# Compose: Training a model to generate music

In [1]:
import os
import pickle
import numpy
from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network

/Users/davidfoster/.virtualenvs/gdl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Set parameters

In [3]:
# run params
section = 'compose'
run_id = '0006'
music_name = 'cello'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])


store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(run_folder):
    os.mkdir(run_folder)
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))
    


mode = 'build' # 'load' # 

# data params
intervals = range(1)
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True

## Extract the notes

In [35]:
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()
        

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.extend(['START'] * seq_len)
            durations.extend([0]* seq_len)

            for element in score.flat:
                
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f) 

35 files in total
1 Parsing data/cello/cs1-2all.mid
2 Parsing data/cello/cs5-1pre.mid
3 Parsing data/cello/cs4-1pre.mid
4 Parsing data/cello/cs3-5bou.mid
5 Parsing data/cello/cs1-4sar.mid
6 Parsing data/cello/cs2-5men.mid
7 Parsing data/cello/cs3-3cou.mid
8 Parsing data/cello/cs2-3cou.mid
9 Parsing data/cello/cs1-6gig.mid
10 Parsing data/cello/cs6-4sar.mid
11 Parsing data/cello/cs4-5bou.mid
12 Parsing data/cello/cs4-3cou.mid
13 Parsing data/cello/cs5-3cou.mid
14 Parsing data/cello/cs6-5gav.mid
15 Parsing data/cello/cs6-6gig.mid
16 Parsing data/cello/cs2-1pre.mid
17 Parsing data/cello/cs3-1pre.mid
18 Parsing data/cello/cs3-6gig.mid
19 Parsing data/cello/cs2-6gig.mid
20 Parsing data/cello/cs2-4sar.mid
21 Parsing data/cello/cs3-4sar.mid
22 Parsing data/cello/cs1-5men.mid
23 Parsing data/cello/cs1-3cou.mid
24 Parsing data/cello/cs6-1pre.mid
25 Parsing data/cello/cs2-2all.mid
26 Parsing data/cello/cs3-2all.mid
27 Parsing data/cello/cs1-1pre.mid
28 Parsing data/cello/cs5-2all.mid
29 Parsing 

## Create the lookup tables

In [37]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [38]:
print('\nnote_to_int')
note_to_int


note_to_int


{'A2': 0,
 'A2.A3': 1,
 'A2.B2': 2,
 'A2.C3': 3,
 'A2.D3': 4,
 'A2.E-3': 5,
 'A2.E3': 6,
 'A2.E3.A3': 7,
 'A2.E3.C#4': 8,
 'A2.E3.C#4.A4': 9,
 'A2.E3.C#4.E4': 10,
 'A2.E3.C#4.G#4': 11,
 'A2.E3.C4': 12,
 'A2.E3.D4': 13,
 'A2.F#3': 14,
 'A2.F#3.C4': 15,
 'A2.F#3.D4': 16,
 'A2.F#3.D4.A4': 17,
 'A2.F#3.D4.E4': 18,
 'A2.F#3.D4.F#4': 19,
 'A2.F3': 20,
 'A2.F3.C4': 21,
 'A2.F3.D4': 22,
 'A2.F3.D4.A4': 23,
 'A2.G3': 24,
 'A2.G3.C#4': 25,
 'A2.G3.D4': 26,
 'A3': 27,
 'A3.B-3': 28,
 'A3.B3': 29,
 'A3.C#4.E4': 30,
 'A3.C4': 31,
 'A3.D4': 32,
 'A3.E4': 33,
 'A3.E4.F#4': 34,
 'A3.E4.G4': 35,
 'A3.F#4': 36,
 'A3.F4': 37,
 'A3.G4': 38,
 'A4': 39,
 'B-2': 40,
 'B-2.A3': 41,
 'B-2.B-3': 42,
 'B-2.D3': 43,
 'B-2.D3.A3': 44,
 'B-2.D3.E-3.G#3': 45,
 'B-2.D3.G#3': 46,
 'B-2.E-3': 47,
 'B-2.E3': 48,
 'B-2.E3.D4': 49,
 'B-2.F#3.C#4.E4': 50,
 'B-2.F3': 51,
 'B-2.F3.C4': 52,
 'B-2.F3.D4': 53,
 'B-2.F3.E-4': 54,
 'B-2.G#3': 55,
 'B-2.G3': 56,
 'B-2.G3.D4': 57,
 'B-3': 58,
 'B-3.C4': 59,
 'B-4': 60,
 'B2': 61,
 

In [39]:
print('\nduration_to_int')
duration_to_int


duration_to_int


{0: 0,
 Fraction(1, 12): 1,
 Fraction(1, 6): 2,
 0.25: 3,
 Fraction(1, 3): 4,
 0.5: 5,
 Fraction(2, 3): 6,
 0.75: 7,
 1.0: 8,
 1.25: 9,
 Fraction(4, 3): 10,
 1.5: 11,
 1.75: 12,
 2.0: 13,
 2.25: 14,
 2.5: 15,
 3.0: 16,
 4.0: 17}

## Prepare the sequences used by the Neural Network

In [42]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [44]:
print('pitch input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('pitch output')
print(network_output[0][0])
print('duration output')
print(network_output[1][0])

pitch input
[386 386 386 386 386 386 386 386 386 386 386 386 386 386 386 386 386 386
 386 386 386 386 386 386 386 386 386 386 386 386 386 386]
duration input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pitch output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Create the structure of the neural network

In [45]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    38700       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    1800        input_2[0][0]                    
__________________________________________________________________________________________________
concatenat

In [46]:
plot_model(model, to_file=os.path.join(run_folder ,'viz/model.png'), show_shapes = True, show_layer_names = True)

## Train the neural network

In [47]:
weights_folder = os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [48]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )



Train on 22376 samples, validate on 5595 samples
Epoch 1/2000000
22376/22376 [==============================] - 107s 5ms/step - loss: 4.2511 - pitch_loss: 3.4720 - duration_loss: 0.7791 - val_loss: 3.9800 - val_pitch_loss: 3.2307 - val_duration_loss: 0.7493
Epoch 2/2000000
22376/22376 [==============================] - 101s 4ms/step - loss: 3.7509 - pitch_loss: 3.1665 - duration_loss: 0.5845 - val_loss: 3.8029 - val_pitch_loss: 3.1389 - val_duration_loss: 0.6640
Epoch 3/2000000
22376/22376 [==============================] - 104s 5ms/step - loss: 3.5442 - pitch_loss: 3.0222 - duration_loss: 0.5220 - val_loss: 3.7914 - val_pitch_loss: 3.0135 - val_duration_loss: 0.7779
Epoch 4/2000000
22376/22376 [==============================] - 100s 4ms/step - loss: 3.3763 - pitch_loss: 2.8962 - duration_loss: 0.4801 - val_loss: 3.6327 - val_pitch_loss: 2.9615 - val_duration_loss: 0.6712
Epoch 5/2000000
22376/22376 [==============================] - 101s 5ms/step - loss: 3.2341 - pitch_loss: 2.7942 - 

Epoch 40/2000000
22376/22376 [==============================] - 101s 5ms/step - loss: 0.5359 - pitch_loss: 0.4968 - duration_loss: 0.0391 - val_loss: 6.5611 - val_pitch_loss: 5.3147 - val_duration_loss: 1.2464
Epoch 41/2000000
22376/22376 [==============================] - 100s 4ms/step - loss: 0.5283 - pitch_loss: 0.4917 - duration_loss: 0.0366 - val_loss: 6.4475 - val_pitch_loss: 5.1770 - val_duration_loss: 1.2705
Epoch 42/2000000
22376/22376 [==============================] - 101s 5ms/step - loss: 0.5055 - pitch_loss: 0.4695 - duration_loss: 0.0360 - val_loss: 6.6874 - val_pitch_loss: 5.4064 - val_duration_loss: 1.2810
Epoch 43/2000000
22376/22376 [==============================] - 99s 4ms/step - loss: 0.4920 - pitch_loss: 0.4561 - duration_loss: 0.0358 - val_loss: 6.5841 - val_pitch_loss: 5.3238 - val_duration_loss: 1.2603
Epoch 44/2000000
22376/22376 [==============================] - 99s 4ms/step - loss: 0.4799 - pitch_loss: 0.4440 - duration_loss: 0.0359 - val_loss: 6.7462 - val

KeyboardInterrupt: 